In [2]:
import sqlite3

conn = sqlite3.connect("pronostico_tiempo.db")
conn.cursor().execute('''CREATE TABLE IF NOT EXISTS PronosticoTiempo (
    Provincia TEXT,
    Temperatura REAL,
    Descripcion TEXT,
    Humedad REAL,
    Viento REAL
)''')
conn.close()


In [12]:
import requests
import sqlite3
import pandas as pd
from tabulate import tabulate

# Reemplaza 'e99f0d4c789ee0d027c1384ced1d9928' con tu clave API
api_key = 'e99f0d4c789ee0d027c1384ced1d9928'

# Coordenadas geográficas de las provincias Argentinasprovincias = [
    {
        "nombre": "Buenos Aires",
        "latitud": -34.61,
        "longitud": -58.38
    },
    {
        "nombre": "Catamarca",
        "latitud": -28.29,
        "longitud": -66.22
    },
    {
        "nombre": "Chaco",
        "latitud": -27.27,
        "longitud": -60.18
    },
    {
        "nombre": "Chubut",
        "latitud": -45.22,
        "longitud": -71.58
    },
    {
        "nombre": "Córdoba",
        "latitud": -31.31,
        "longitud": -64.18
    },
    {
        "nombre": "Corrientes",
        "latitud": -27.46,
        "longitud": -58.54
    },
    {
        "nombre": "Entre Ríos",
        "latitud": -32.32,
        "longitud": -58.38
    },
    {
        "nombre": "Formosa",
        "latitud": -26.11,
        "longitud": -60.18
    },
    {
        "nombre": "Jujuy",
        "latitud": -24.01,
        "longitud": -65.25
    },
    {
        "nombre": "La Pampa",
        "latitud": -37.27,
        "longitud": -64.38
    },
    {
        "nombre": "La Rioja",
        "latitud": -29.29,
        "longitud": -66.22
    },
    {
        "nombre": "Mendoza",
        "latitud": -33.29,
        "longitud": -68.38
    },
    {
        "nombre": "Misiones",
        "latitud": -27.11,
        "longitud": -54.54
    },
    {
        "nombre": "Neuquén",
        "latitud": -38.27,
        "longitud": -70.38
    },
    {
        "nombre": "Río Negro",
        "latitud": -41.27,
        "longitud": -68.38
    },
    {
        "nombre": "Salta",
        "latitud": -24.01,
        "longitud": -67.22
    },
    {
        "nombre": "San Juan",
        "latitud": -31.29,
        "longitud": -68.38
    },
    {
        "nombre": "San Luis",
        "latitud": -33.29,
        "longitud": -65.38
    },
    {
        "nombre": "Santa Cruz",
        "latitud": -46.22,
        "longitud": -70.38
    },
    {
        "nombre": "Santa Fe",
        "latitud": -32.32,
        "longitud": -60.54
    },
    {
        "nombre": "Santiago del Estero",
        "latitud": -27.29,
        "longitud": -64.22
    },
    {
        "nombre": "Tierra del Fuego",
        "latitud": -54.22,
        "longitud": -68.38
    },
    {
        "nombre": "Tucumán",
        "latitud": -26.51,
        "longitud": -65.12
    }
]

# Lista para almacenar los datos de pronóstico del tiempo
datos_provincias = []

# Crea el archivo de la base de datos
with sqlite3.connect("pronostico_tiempo.db") as conn:
    conn.execute("CREATE TABLE IF NOT EXISTS PronosticoTiempo (Provincia TEXT, Temperatura REAL, Descripcion TEXT, Humedad REAL, Viento REAL)")

for provincia in provincias:
    url = f'https://api.openweathermap.org/data/2.5/weather?lat={provincia["latitud"]}&lon={provincia["longitud"]}&appid={api_key}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        temperatura_kelvin = data['main']['temp']
        descripcion = data['weather'][0]['description']
        humedad = data['main']['humidity']
        velocidad_viento = data['wind']['speed']

        # Convierte la temperatura de Kelvin a Celsius
        temperatura_celsius = temperatura_kelvin - 273.15

        datos_provincias.append({
            "Provincia": provincia["nombre"],
            "Temperatura (°C)": round(temperatura_celsius, 2),
            "Descripción": descripcion,
            "Humedad (%)": humedad,
            "Velocidad del Viento (m/s)": round(velocidad_viento, 2)
        })

# Guardar la información en la base de datos
df = pd.DataFrame(datos_provincias)
df.to_sql("PronosticoTiempo", conn, if_exists="replace")

# Consulta SQL para obtener los datos de la tabla PronosticoTiempo
conn = sqlite3.connect("pronostico_tiempo.db")
consulta = "SELECT * FROM PronosticoTiempo"
df = pd.read_sql_query(consulta, conn)

# Muestra los datos en forma de tabla utilizando tabulate
table = tabulate(df, headers='keys', tablefmt='pretty', showindex=False)
print(table)

# Cierra la conexión a la base de datos
conn.close()


+-------+---------------------+------------------+------------------+-------------+----------------------------+
| index |      Provincia      | Temperatura (°C) |   Descripción    | Humedad (%) | Velocidad del Viento (m/s) |
+-------+---------------------+------------------+------------------+-------------+----------------------------+
|   0   |    Buenos Aires     |      18.67       | overcast clouds  |     46      |            6.06            |
|   1   |      Catamarca      |      24.03       |    clear sky     |     20      |            6.2             |
|   2   |        Chaco        |      19.22       | overcast clouds  |     98      |            3.21            |
|   3   |       Chubut        |       3.23       |    light rain    |     87      |            7.57            |
|   4   |       Córdoba       |      25.23       |    clear sky     |     41      |            3.72            |
|   5   |     Corrientes      |      23.84       |    light rain    |     98      |            2